In [1]:
%reload_ext autoreload
%autoreload 2

from utils import *

In [2]:
fpath = 'qa_corpus.csv'

In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
content = pd.read_csv(fpath, encoding='utf-8')

In [5]:
content.head()

,qid,question,answer
0,1,补发网银盾,您可以到智慧柜员机办理此项业务，方便快捷，节省您的宝贵时间哦。若去柜台办理，请刷身份证取号。
1,2,代发工资,您要办理的是对公开户、销户、签约及其他业务。请刷身份证，小龙人来帮您取个号吧。
2,3,对帐单查询打印,您要办理的是对公开户、销户、签约及其他业务。请刷身份证，小龙人来帮您取个号吧。
3,4,理财产品取号,请说出您需要办理的业务
4,5,密码修改,请您带上身份证到自助柜员机办理，简单快捷，操作容易哦。


In [6]:
import jieba
import re


import os
LTP_DATA_DIR = 'D:/ProgramData/nlp_package/ltp_v34'  # ltp模型目录的路径
cws_model_path = os.path.join(LTP_DATA_DIR, 'cws.model')  # 分词模型路径，模型名称为`cws.model`

from pyltp import Segmentor
segmentor = Segmentor()  # 初始化实例
segmentor.load_with_lexicon(cws_model_path, 'lexicon_seg.txt') # 加载外部词典文件路径


def token(string):
    return re.findall(r'[\d|\w]+', string)

def cut(string): 
    return ' '.join(segmentor.segment(string))

def filter_text(content):
    q_content = content['question'].tolist()
    q_content = [token(str(n)) for n in q_content]
    q_content = [' '.join(n) for n in q_content]
    q_content = [cut(n) for n in q_content]
    return q_content

In [ ]:
segmentor.release()  # 释放模型

In [ ]:
# from pyhanlp import *

# tokenizer = JClass("com.hankcs.hanlp.seg.NShort.NShortSegment")
# tokenizer2 = JClass("com.hankcs.hanlp.tokenizer.NLPTokenizer")
# tokenizer3 = JClass('com.hankcs.hanlp.tokenizer.StandardTokenizer')


In [7]:
q_content = filter_text(content)

In [8]:
q_content[21231]

'如何 注销 E 商贸通'

In [9]:
# for text in q_content:
#     if '办理' in text:
#         print(text)

In [10]:
stopwords = []
with open('chinese_stopwords.txt', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        if len(line.strip()) < 2:
            stopwords.append(line.strip())

with open('哈工大停用词表.txt', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        if len(line.strip()) < 2:
            stopwords.append(line.strip())

### tf-idf



In [11]:
# 注意，自定义了analyzer
vectorized = TfidfVectorizer(max_features=12000, tokenizer=lambda x: x.split(), stop_words=stopwords)
X = vectorized.fit_transform(q_content)

print(len(vectorized.vocabulary_))
print(X.shape)

11133
(32526, 11133)


D:\ProgramData\envs\nlpLearning\lib\site-packages\sklearn\feature_extraction\text.py:286: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['δ', 'ψ', 'в', 'ⅲ', 'ａ'] not in stop_words.
  sorted(inconsistent))


In [12]:
import scipy.sparse as sp


X_array = X.toarray()
X_array = sp.csr_matrix(X_array)

In [13]:
# word_2_id = vectorized.vocabulary_

# for word, i in word_2_id.items():
#     if len(word) < 2:
#         print(word)

In [14]:
import numpy as np


np.nonzero(X[100].toarray()[0])

(array([ 5278,  9413,  9499, 10423], dtype=int64),)

In [15]:
from scipy.spatial.distance import cosine

def distance(v1, v2): return cosine(v1, v2)

In [16]:
def show_diff(i1, i2):
    print(content.question.iloc[i1])
    print(content.question.iloc[i2])

    print(distance(X_array[i1].toarray()[0], X_array[i2].toarray()[0]))


In [17]:
show_diff(1212, 1211)

我可以买东西吗
我要买东西
0.10267861409497203


In [18]:
show_diff(21231, 21230)

如何注销E商贸通
如何办理E商贸通出入金
0.377988279224442


In [19]:
show_diff(21231, 21211)

如何注销E商贸通
外汇汇款手续费
1.0


### BM25

In [20]:
from collections import Counter
from math import log

def get_tf(sent_i, sent_j):
    """计算bm25的term frequence. sent来自预处理的sent_of_words列表。"""
    freq = {}
    sent_i_counts = Counter(sent_i)
    # 计算i句中的词，在j句中的tf
    for w in sent_j:
        # if not self.filte_words(w_item):
        #     continue
        if w in sent_i_counts:
            freq[w] = sent_i_counts[w]
        else:
            freq[w] = 0
    total = len(sent_i)
    return {word: count / total for word, count in freq.items()}


def get_idf(content):
    """计算inverse document frequence. 这里计算句子的相似度，所以计算inverse sentence frequence"""
    total_sent = len(content) + 1 # 假设有一个句子包含所有词
    avg_len = 0
    doc_freq = {}
    for sent in content:
        sent = sent.lower().split()
        avg_len += len(sent)
        words = list({w for w in sent})
        for word in words:
            # 假设有一个句子包含所有词
            count = doc_freq.setdefault(word, 1)
            doc_freq[word] = count + 1
    avg_len /= total_sent
    # sklearn中的实现方式
    idf = {word: log(total_sent / df) + 1 for word, df in doc_freq.items()}
    return idf, avg_len


def sent_corelation_func(sent_i, sent_j, idf, avg_len, k1=1.5, b=0.75):
    """计算bm25。

    sent_i ： 与query对比的句子，在文档中进行遍历，D
    sent_j : query的句子, Q
    """
    tf = get_tf(sent_i, sent_j)

    K = k1 * (1 - b + b * len(sent_i) / avg_len)
    bm25 = 0
    for j_word in sent_j:
        bm25 += idf[j_word] * tf[j_word] * (k1 + 1) / (tf[j_word] + K)
    return bm25

In [21]:
idf, avg_len = get_idf(q_content)
sent_corelation_func(q_content[21231].lower().split(), q_content[21230].lower().split(), idf, avg_len)

8.148892425018543

In [22]:
len(idf)

11320

### 编辑距离

### Simhash

### Sentence Embedding

### fasttext

In [23]:
import fasttext

In [24]:
model = fasttext.load_model("cc.zh.300.bin")

# del model
# model.get_word_vector('学生惠')

使用fb官方模型，在任务数据上fine tuning。使用gensim进行训练。

In [31]:
from gensim.models.fasttext import load_facebook_model

fb_model  = load_facebook_model('cc.zh.300.bin')

In [32]:
fb_model.estimate_memory()

{'vocab': 1000000000,
 'syn0_vocab': 2400000000,
 'syn1neg': 2400000000,
 'syn0_ngrams': 1474220400,
 'buckets_word': 121864840,
 'total': 7396085240}

In [35]:
new_sent = list(map(lambda s: s.lower().split(), q_content))
len(new_sent)

fb_model.build_vocab(new_sent, update=True)
fb_model.train(sentences=new_sent, total_examples=len(new_sent), epochs=5)

In [45]:
fb_model.wv.similarity('储蓄卡','信用卡')
fb_model.wv.similarity('开户','办理')
fb_model.wv.similarity('贷款','存款')

0.7010534

In [43]:
selected = []
for w in word_2_id.keys():
    vec = fb_model.wv[w]
    selected.append((w, vec))

In [44]:
len(selected)

11133

In [61]:
with open('selected_words_fasttext_300.txt', 'w', encoding='utf-8') as f:
    first_line = str(len(selected)) + '\t' + str(300)
    f.write(first_line)
    for w, v in selected:
        line =  '\n' + str(w) + '\t' + array_to_string(v)
        f.write(line)

In [ ]:
del fb_model

### tencent wordvec

oov的词，包括该场景下的重要词汇，比如 联名卡， 溢缴款， 学生惠

In [172]:
# from gensim.models import KeyedVectors

# model = KeyedVectors.load_word2vec_format("100000-small.txt")

In [1]:
import io

def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = map(float, tokens[1:])
    return data

In [2]:
data = load_vectors('sgns.target.word-character.char1-2.dynwin5.thr10.neg5.dim300.iter5')

In [3]:
len(data)

636036

In [17]:
selected = []
for w in word_2_id.keys():
    try:
        vec = data[w]
        selected.append((w, vec))
    except KeyError:
        continue

In [18]:
'联名卡' in selected

False

In [19]:
len(selected)

9316

In [ ]:
del data
del selected

### sentence vec 表示方法

短文本应该有较好的变现力

In [67]:
selected = {w: v for w, v in selected}

In [84]:
def sent_to_vec(sent):
    res = 0
    length = 0
    for w in sent:
        try:
            res += selected[w]
            length += 1
        except KeyError:
            continue
    return res / length

In [76]:
import numpy as np

def search_sentvec(query):
    """"""
    words = filter_text(query)
    postags = postagger.postag(words)

    to_and = []
    for i, postag in enumerate(postags):
        if postag in and_pos_set:
            to_and.append(words[i])
    print(to_and)
    
    query_vec = words
    try:
        candidates_ids = [word_2_id[w] for w in to_and]
    except KeyError:
        pass

    documents_ids = [
         set(np.nonzero(inverse_idx[_id].toarray()[0])[0]) for _id in candidates_ids
    ]

    merged_documents = reduce(and_, documents_ids)
    sorted_docuemtns_id = sorted(merged_documents, key=lambda i: distance(sent_to_vec(query_vec), sent_to_vec(q_content[i].lower().split())))

    return sorted_docuemtns_id

In [85]:
search_sentvec("办理龙卡")

['办理龙卡']
['办理', '龙卡']


[344,
 21590,
 268,
 20460,
 20038,
 30928,
 30939,
 32362,
 30914,
 20563,
 30934,
 20942,
 21511,
 21006,
 20446,
 20035,
 22721,
 20617,
 22604,
 22636,
 22639,
 32090,
 32202,
 31033,
 22722,
 30938,
 21122,
 30941,
 32363,
 22635,
 22119,
 22640,
 20570,
 22368,
 20390,
 20023,
 23133,
 22365]

In [90]:
content.iloc[22721]

qid                                                     55052
question                                    办理龙卡随付贷业务能否累积消费积分
answer      办理龙卡随付贷业务可以累积消费积分。使用分期付款方式同样享有消费积分，根据刷卡分期金额一次性...
Name: 22721, dtype: object

### StarSpace
词向量训练 https://github.com/facebookresearch/StarSpace
此数据集由于没有tag标注，也没有相似集合分类的标签，完全的无监督数据，所以没哟采用这个方法。


### DSSM、Siamese
没有相似性标签


### 直接使用无监督的分布式特征表示
Bert encoding或者各种word2vec的表示

---

如果有更进一步的标注，可以采用更多的方法。这3万条数据，自己手动标注是不现实的。

## 倒排索引

In [16]:
word_2_id = vectorized.vocabulary_
id_2_word = {d: w for w, d in word_2_id.items()}

inverse_idx = X_array.transpose()
print(inverse_idx.shape)

(11133, 32526)


In [71]:
inverse_idx.size

113547

In [72]:
from operator import and_
from functools import reduce


pos_model_path = os.path.join(LTP_DATA_DIR, 'pos.model')

from pyltp import Postagger
postagger = Postagger() # 初始化实例
postagger.load_with_lexicon(pos_model_path, 'lexicon.txt')  # 加载模型

and_pos_set = {'n', 'v', 'm', 'nh', 'ni', 'nl', 'ns', 'nt', 'ws'}


def token(string):
    return re.findall(r'[\d|\w]+', string)

def cut(string): 
    return segmentor.segment(string)

def filter_text(string):
    q_content = token(string.lower())
    print(q_content)
    q_content = ' '.join(q_content)
    q_content = cut(q_content)
    return q_content


# segmentor
def search(query):
    """"""
    words = filter_text(query)
    postags = postagger.postag(words)

    to_and = []
    for i, postag in enumerate(postags):
        if postag in and_pos_set:
            to_and.append(words[i])
    print(to_and)
    
    query_vec = vectorized.transform([' '.join(words)]).toarray()
    try:
        candidates_ids = [word_2_id[w] for w in to_and]
    except KeyError:
        pass

    documents_ids = [
         set(np.nonzero(inverse_idx[_id].toarray()[0])[0]) for _id in candidates_ids
    ]

    merged_documents = reduce(and_, documents_ids)
    
    sorted_docuemtns_id = sorted(merged_documents, key=lambda i: distance(query_vec, X_array[i].toarray()[0]))

    return sorted_docuemtns_id

In [ ]:
search("办理龙卡")

In [ ]:
content.iloc[344]

In [ ]:
content.iloc[22365]

In [ ]:
def search_bm25(query):
    """"""
    words = filter_text(query)
    postags = postagger.postag(words)

    to_and = []
    for i, postag in enumerate(postags):
        if postag in and_pos_set:
            to_and.append(words[i])
    print(to_and)
    
    query_vec = words
    try:
        candidates_ids = [word_2_id[w] for w in to_and]
    except KeyError:
        pass

    documents_ids = [
         set(np.nonzero(inverse_idx[_id].toarray()[0])[0]) for _id in candidates_ids
    ]

    merged_documents = reduce(and_, documents_ids)
    sorted_docuemtns_id = sorted(merged_documents, key=lambda i: sent_corelation_func(query_vec, q_content[i].lower().split(), idf, avg_len), reverse=True)

    return sorted_docuemtns_id

In [86]:
search_bm25("之前我已经通过上行短信办理信用卡还款，我再次通过上行短信办理信用卡还款还需要输入信用卡全账号吗？")

['之前我已经通过上行短信办理信用卡还款', '我再次通过上行短信办理信用卡还款还需要输入信用卡全账号吗']
['上行', '短信', '办理', '信用卡', '还款', '上行', '短信', '办理', '信用卡', '还款', '需要', '输入', '信用卡', '全账号']


[31469, 31470]

In [ ]:
content.iloc[]

In [102]:
search("怎么办医保联名卡")

['怎么办医保联名卡']
['联名卡']


[31922,
 31924,
 31923,
 31919,
 24241,
 24331,
 24636,
 520,
 31920,
 265,
 61,
 673,
 22461,
 20945,
 757,
 268,
 31247,
 20941,
 20942,
 22679,
 20940,
 31921,
 392,
 21457,
 21461,
 31925,
 21460,
 21462,
 20944,
 21768,
 21458,
 21459,
 266,
 30813,
 21842,
 20348]

In [103]:
content.iloc[31924]

qid                                                     66591
question                                           如何申请医保联名卡？
answer      医保联名卡由我行与各地医保管理部门签署联合发行协议，由参保人到指定地点按照社会保障和我行相关...
Name: 31924, dtype: object

### 聚类

In [104]:
from pyhanlp import *

ClusterAnalyzer = JClass('com.hankcs.hanlp.mining.cluster.ClusterAnalyzer')

In [138]:
analyzer = ClusterAnalyzer()

for idx, text in enumerate(q_content):
    words = text.lower().split()
    postags = postagger.postag(words)

    to_and = []
    for i, postag in enumerate(postags):
        if postag in and_pos_set:
            to_and.append(words[i])

    analyzer.addDocument(idx, ', '.join(text.lower().split()))

In [151]:
res = list(analyzer.repeatedBisection(10))

In [152]:
len(res)

10

In [154]:
for i in res[0]:
    print(q_content[i])

hone 手机 有 什么 要求 吗
请问 为什么 我 在 使用 客户端 登录 手机 银行 时 有时 提示会 话 超时
怎样 使用 客户端 修改 手机 银行 的 登录 密码
什么 是 客户端 手机 银行
通过 客户端 如何 开通 手机 银行 功能
如何 知道 我 手机 的 固件 版本
如何 将 我 的 手机 银行 解除 绑定 Android 手机
Android 手机 能否 使用 WAP 版 手机 银行
怎么 呢
你 又 不 知道 了
我 可以 变更 接收 理财 资讯 短信 服务 的 手机 号码 吗
我 能 单独 变更 贵 金属 行情 提醒 的 而 不 变更 基金 净值 的 短信 接收 手机 吗
嗯 好 的
我 可以 用 多 个 手机 短信 签约 基金 短信 服务 吗
是 怎么 着 了
为什么 我 无法 在 网站 上 修改 我 的 主手机 号码 呢
没 发现
通过 网站 变更 主 手机 号码 时 可以 输入 原 副 手机 号码 吗
又
银行 本票 提示 付款 时间
我 可以 通过 电话 银行 注销 账户 变动 通知 服务 吗
我们 那个
我 可以 通过 手机 银行 注销 账户 变动 通知 服务 吗
我 通过 手机 银行 注销 短信通 服务 的 账户 必须 是 手机 银行 账户 么
我 通过 手机 银行 注销 账户 变动 通知 服务 的 账户 必须 是 手机 银行 账户 么
为什么 我 用 手机 银行 客户端 注销 短信 金融 服务 时 需要 输入 短信 验证码
我 编辑 短信 注销 服务 怎么 系统 提示 失败
要 勇于 创新 知道 不
独立 注销 短信 转账 及 短信 缴费 功能 的 手机 号码 可以 副 手机 号码 吗
我 要 知道
独立 取消 账户 变动 通知 服务 可以 使用 副 手机 号码 注销 吗
要 怎么 才 能 用 你
外汇 买卖 怎么 操作
我 注销 短信 转账 时 为什么 我 的 副 手机 号码 收 不 到 注销 提醒 短信
怎么 操作 外汇 买卖
我 在 手机 银行 上 开通 短信 转账 可以 去 柜台 维护 和 注销 吗
我 的 信用卡 刚刚 消费 了 300 元 怎么 没 收到 短信
咱俩 几 天 没 聊 了
如何 查询 外汇 汇款 的 银行 SWIFT CODE
你 知道
怎么 自助 办理 结售汇
为什么 我 设定 的 自动 转 存 没有 

通过聚类来分离不同的intent的数据，误差不可避免，并且难以评测实际的效果，不建议使用。因为作为下游任务输入，产生了未知的固有偏差。

只使用聚类来缩小搜索范围

In [28]:
import io

def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split('\t')
        data[tokens[0]] = np.array(list(map(float, tokens[1].split())))
    return data

In [30]:
selected = load_vectors('selected_words_fasttext_300.txt')

In [31]:
sent_avg_vec(q_content[1].lower().split(), selected, idf=idf)

array([ 1.65208613e+00,  1.18255034e+00,  4.24145704e+00, -1.59712788e+00,
        8.60255272e-01,  3.01796617e+00, -1.37390461e+00, -7.18926022e-01,
       -7.14630671e-01,  3.52789671e-01, -2.04633439e-01, -5.74183184e-01,
        1.19527118e+00,  2.80558763e+00, -8.11127023e-01,  3.28578360e-01,
       -2.57038264e+00,  1.33348900e+00, -2.15569896e+00,  1.00758357e+00,
       -4.23534664e+00, -9.87053533e-01, -9.37054116e-01, -8.58435405e-01,
        6.35899910e-01, -3.09533095e+00,  1.12948356e+00,  4.09486714e-01,
       -1.73818548e+00,  8.27988133e-01,  4.58147857e-01, -7.87355935e-01,
       -1.25655238e+00,  1.58097459e+00, -2.10624873e+00, -1.15538069e+00,
        3.72786165e+00, -2.61324533e+00, -1.06625954e+00,  1.02676801e+00,
       -3.02137220e-01, -9.77209518e-01,  5.35068637e-01,  1.96771122e+00,
        1.47119064e+00, -8.84401688e-01,  2.26167245e+00, -2.15688232e-01,
        6.03710012e-01, -2.62562389e-01, -1.58422013e-01,  6.26440330e-01,
        1.24142646e+00,  

In [32]:
X = []
chit_idx = []
for i in range(len(q_content)):
    try:
        X.append(sent_avg_vec(q_content[i].lower().split(), selected, idf=idf))
    except Exception:
        # print(i, ' ', content.question[i])
        chit_idx.append(i)
        continue

In [33]:
len(X)

31976

In [34]:
X = np.array(X)

In [35]:
A = np.array([[1, 2], [1, 4], [1, 0], [10, 2], [10, 4], [10, 0]])

In [40]:
from spherecluster import SphericalKMeans
skm = SphericalKMeans(n_clusters=)
skm.fit(X)

SphericalKMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=2,
        n_init=10, n_jobs=1, normalize=True, random_state=None, tol=0.0001,
        verbose=0)

In [54]:
import random

idx = random.sample(range(0, 10000), 10)

In [58]:
def test(idx):
    for i in idx:
        print(q_content[i])
        try:
            print(skm.predict(sent_avg_vec(q_content[i].lower().split(), selected, idf=idf).reshape(1, -1)))
        except Exception:
            continue

In [57]:
test(idx)

你 到底 在 做 什么
[1]
你 其实 挺笨 的
[1]
你 可 不 可以 陪 我 打蓝球 啊
[1]
ATM CRS 还 信用卡 到账 时间
[0]
那 我 应该 怎么 办 呢
[1]
我 现在 不 能 接听 电话
[0]
随身 携带
[0]
过 得 好 吗
辛苦 啦
[1]
问 你 个 问题 请问 世界 上 什么 鼠用 2 只 脚 走路 回答 袋鼠 不 对
[1]


In [63]:
from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering

In [59]:
kmeans = KMeans(n_clusters=2, random_state=0).fit(X)

In [64]:
sc = SpectralClustering(n_clusters=2, random_state=0).fit(X)

KeyboardInterrupt: 

In [60]:
def test(idx):
    for i in idx:
        print(q_content[i])
        try:
            print(kmeans.predict(sent_avg_vec(q_content[i].lower().split(), selected, idf=idf).reshape(1, -1)))
        except Exception:
            continue

In [61]:
test(idx)

你 到底 在 做 什么
[0]
你 其实 挺笨 的
[0]
你 可 不 可以 陪 我 打蓝球 啊
[0]
ATM CRS 还 信用卡 到账 时间
[1]
那 我 应该 怎么 办 呢
[0]
我 现在 不 能 接听 电话
[1]
随身 携带
[1]
过 得 好 吗
辛苦 啦
[0]
问 你 个 问题 请问 世界 上 什么 鼠用 2 只 脚 走路 回答 袋鼠 不 对
[0]


In [143]:
import sklearn as skl

In [144]:
skl.__version__

'0.22.2'

In [145]:
del kmeans